In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import partial
from sklearn.model_selection import train_test_split, GridSearchCV, ShuffleSplit
from sklearn.svm import SVR, SVC

#torch.manual_seed(100)
np.random.seed(100)

In [2]:
# load prepprocessed data


In [3]:
def _create_evaluator(estimator, param_grid, scoring, cv=4, N=5, callback=None):
    
    gs_estimator = GridSearchCV(estimator=estimator, param_grid=param_grid, scoring=scoring, cv=cv, n_jobs=-1, refit=True)
    
    def evaluate(X, y, verbose=True):
        
        test_losses = np.zeros(N)
        
        for n in range(N):
            
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=n)
            gs_estimator.fit(X_train, y_train)
            test_loss = -gs_estimator.score(X_test, y_test)
            test_losses[n] = test_loss
            
            if callback is not None: callback(gs_estimator, X_test, y_test)
            if verbose: print('Iteration {:02d} | Test Loss = {:0.4f}'.format(n, test_loss))

        return np.mean(test_losses), np.std(test_losses)

    return evaluate

In [4]:
def _load_tool(tool_length):
    
    npzfile=np.load(f'../preprocessed/neutouch_tool_{tool_length}.npz')
    
    X = npzfile['signals'] / 100
    y = npzfile['labels'] * 100
    
    X = np.abs(np.fft.fft(X)) / 10
    X = np.reshape(X, (X.shape[0], -1))
    
    print('Max,Min:', X.max(), X.min())

    return X, y

In [5]:
def _evaluate_tool_svm(tool_length, signal_type, transformation, kernel, frequency):

    X, y = _load_tool(tool_length)
    
    param_grid = { 'C': [1, 3, 10, 30, 100] }
    
    estimator = SVR(kernel=kernel, max_iter=5000)
    evaluate = _create_evaluator(estimator, param_grid, 'neg_mean_absolute_error')
    
    return evaluate(X, y)

In [6]:
my_model = partial(_evaluate_tool_svm, 50, 'neutouch', 'default', 'rbf')

In [ ]:
my_model(frequency=4000)

Max,Min: 0.41255762992937806 0.0
